In [26]:
import json
import re
import json
import os
import re
import os
from datetime import datetime
#pip install "gpt4all[cuda]"

In [27]:
#path = 'C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones/56507943/transcriptions/'

In [28]:
def convertir_json_a_texto_con_tiempo(ruta_archivo_json):
    # Cargar el archivo JSON
    with open(ruta_archivo_json, "r", encoding="utf-8") as file:
        data = json.load(file)

    # Inicializar variables
    primer_timestamp = None
    transcripciones = []

    # Procesar transcripciones
    for transcript in data.get("transcripts", []):
        for phrase in transcript.get("phrases", []):
            speaker = "Ejecutivo" if phrase.get("participantPurpose") == "internal" else "Cliente"
            text = phrase.get("text", "").strip()
            start_time_ms = phrase.get("startTimeMs", 0)

            if primer_timestamp is None:
                primer_timestamp = start_time_ms

            tiempo_normalizado_ms = start_time_ms - primer_timestamp
            minutos = (tiempo_normalizado_ms // 1000) // 60
            segundos = (tiempo_normalizado_ms // 1000) % 60
            tiempo_formateado = f"{minutos:02}:{segundos:02}"

            if text:
                transcripciones.append(f"[{tiempo_formateado}] {speaker}: {text}")

    transcripcion_procesada = "\n".join(transcripciones)

    # Generar ruta de salida reemplazando la extensión con "_transcripcion_texto.txt"
    ruta_salida = os.path.splitext(ruta_archivo_json)[0] + "_texto.txt"

    # Guardar resultado
    with open(ruta_salida, "w", encoding="utf-8") as file:
        file.write(transcripcion_procesada)

    print(f"✅ Transcripción corregida y guardada en: {ruta_salida}")

    return ruta_salida

# Ejemplo de uso:
#ruta_transcripcion_texto = convertir_json_a_texto_con_tiempo(path+"transription_0.json")

In [29]:
# Lista de frases comunes de grabadora que queremos eliminar
frases_grabadora = [
    "tu llamada es importante para nosotros",
    "presentamos un alto flujo de llamadas",
    "te agradecemos permanecer en línea",
    "espera de atención de un ejecutivo",
    "nunca te llamaremos para pedir tus claves",
    "nunca te enviaremos link por correo",
    "ingresar a tu banca en línea",
]

def eliminar_frases_grabadora_archivo(ruta_archivo_entrada, frases_eliminar):
    # 📌 Función interna que realiza el filtrado de frases
    def eliminar_frases(texto, frases):
        lineas = texto.split("\n")
        lineas_filtradas = []

        for linea in lineas:
            # Elimina timestamp y etiquetas iniciales si existen (formato flexible)
            contenido = re.sub(r"^\[?\d{0,2}:\d{0,2}\]?\s?(Ejecutivo|Cliente)?\s?:?", "", linea).strip()

            if not any(frase in contenido.lower() for frase in frases):
                lineas_filtradas.append(linea)

        return "\n".join(lineas_filtradas)

    # Leer archivo original
    with open(ruta_archivo_entrada, "r", encoding="utf-8") as file:
        transcripcion = file.read()

    # Eliminar frases grabadora
    transcripcion_limpia = eliminar_frases(transcripcion, frases_eliminar)

    # Generar archivo de salida automáticamente
    ruta_archivo_salida = os.path.splitext(ruta_archivo_entrada)[0] + "_sin_grabadora.txt"

    # Guardar el resultado limpio
    with open(ruta_archivo_salida, "w", encoding="utf-8") as file:
        file.write(transcripcion_limpia)

    print(f"✅ Transcripción limpia guardada en: {ruta_archivo_salida}")
    return ruta_archivo_salida

# Ejemplo de cómo usar esta función:
#ruta_resultado_sin_grabadora = eliminar_frases_grabadora_archivo(ruta_transcripcion_texto, frases_grabadora)


In [30]:
import nltk
from nltk.corpus import stopwords
import os

def limpiar_stopwords_transcripcion(ruta_archivo_entrada):
    # Descargar stopwords de NLTK (solo la primera vez)
    nltk.download("stopwords", quiet=True)

    # Cargar stopwords en español
    stop_words = set(stopwords.words("spanish"))

    # Lista personalizada para conservar palabras importantes en análisis de fraude
    stopwords_permitidas = {"si", "sí", "no", "tarjeta", "clave", "fraude", "banco", "dinero", 
                            "transferencia", "pago", "cuenta", "solamente", "solo", "sola", "solas", 
                            "solos", "mi"}
    
    # Remover las permitidas de la lista original
    stop_words = stop_words - stopwords_permitidas

    # Función interna para limpiar texto
    def limpiar_texto(texto, stop_words):
        palabras = texto.split()
        palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stop_words]
        return " ".join(palabras_filtradas)

    # Cargar archivo original
    with open(ruta_archivo_entrada, "r", encoding="utf-8") as file:
        transcripcion = file.read()

    # Aplicar limpieza
    transcripcion_limpia = limpiar_texto(transcripcion, stop_words)

    # Crear ruta de salida automáticamente
    ruta_salida = os.path.splitext(ruta_archivo_entrada)[0] + "_sin_stopwords.txt"

    # Guardar resultado limpio
    with open(ruta_salida, "w", encoding="utf-8") as file:
        file.write(transcripcion_limpia)

    print(f"✅ Transcripción sin stopwords guardada en: {ruta_salida}")

    return ruta_salida

# Ejemplo de cómo usar la función:
#ruta_resultado_sin_stopwords = limpiar_stopwords_transcripcion(ruta_resultado_sin_grabadora)


In [31]:
import re
import os

def buscar_entrega_con_contexto(ruta_entrada, contexto):
    """
    Busca palabras relacionadas con 'entregar' o 'compartir' y extrae segmentos con contexto.

    Parámetros:
    - ruta_entrada (str): Ruta al archivo de texto de entrada.
    - contexto (int): Número de caracteres antes y después del match.

    Retorna:
    - ruta_salida (str): Ruta del archivo generado.
    - resultados (list): Lista de segmentos encontrados.
    """
    with open(ruta_entrada, "r", encoding="utf-8") as file:
        texto = file.read()

    # Patrón para buscar entregó, entrega, entregado, compartí, compartió, etc.
    patron = re.compile(r"\b(entreg[a-záéíóú]*|compart[a-záéíóú]*)\b", re.IGNORECASE)

    resultados = []
    for match in patron.finditer(texto):
        inicio = max(match.start() - contexto, 0)
        fin = min(match.end() + contexto, len(texto))
        segmento = texto[inicio:fin].strip()
        resultados.append(segmento)

    # Guardar el resultado
    carpeta = os.path.dirname(ruta_entrada)
    ruta_salida = os.path.join(carpeta, "segmentos_entrega_contexto.txt")
    with open(ruta_salida, "w", encoding="utf-8") as file:
        file.write("\n---\n".join(resultados))

    print(f"✅ Segmentos guardados en: {ruta_salida}")
    return ruta_salida, resultados

#ruta_salida, segmentos = buscar_entrega_con_contexto(ruta_resultado_sin_stopwords,200)

In [32]:
from datetime import datetime
from gpt4all import GPT4All

def analizar_clave_con_gpt4all(ruta_transcripcion):
    """
    Analiza si un cliente entregó claves a un tercero usando un modelo GPT4All.

    Retorna:
    - respuesta generada por el modelo (texto)
    """

    ruta_modelo = "C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/mistral-7b-instruct-v0.2.Q4_K_M.gguf"    
    # 📌 Inicializar el modelo
    model = GPT4All(ruta_modelo, device='cuda')

    # 📌 Cargar la transcripción
    with open(ruta_transcripcion, 'r', encoding='utf-8') as archivo:
        transcripcion = archivo.read()

    # 📌 Mostrar el largo en caracteres
    print(f"Largo de la transcripción: {len(transcripcion)} caracteres")

    # 📌 Formatear el prompt
    prompt = f"""
    [INST]
    Eres un analista experto en detección de fraude bancario. Tu tarea es revisar con mucho cuidado la conversación telefónica entre un EJECUTIVO y un CLIENTE que presenta un reclamo por fraude.

    Debes determinar estrictamente si el CLIENTE reconoce o indica explícitamente haber entregado, compartido o revelado cualquier clave, contraseña, código o información confidencial (directa o indirectamente) a alguna persona (incluyendo claramente familiares, terceros, conocidos o desconocidos).

    IMPORTANTE: Si el cliente menciona explícitamente haber entregado claves o información confidencial a alguna persona, aunque sea un familiar (como "mi hija", "mi hijo", "mi esposo/a", etc.), debes responder obligatoriamente "SÍ". 

    Instrucciones para tu respuesta:
    - Primera línea: escribe únicamente "SÍ" o "NO".
    - Segunda línea: Explica brevemente la razón específica basándote literalmente en lo que dijo el cliente en la conversación (usa citas textuales del cliente).

    CONVERSACIÓN:
    {transcripcion}

    RESPUESTA:
    [/INST]
    """
    
    print("🕒 Comienzo del análisis:", datetime.now())

    # 📌 Generar respuesta
    respuesta = model.generate(prompt).strip()
    return respuesta


In [33]:
def extraer_respuesta_binaria(respuesta_modelo):
    """
    Extrae si la respuesta del modelo comienza con "SÍ" o "NO".

    Parámetros:
    - respuesta_modelo (str): Texto completo generado por el modelo.

    Retorna:
    - "SI" o "NO" (en mayúsculas) según la primera línea de la respuesta.
    """
    primera_linea = respuesta_modelo.strip().splitlines()[0].strip().upper()
    
    if primera_linea.startswith(("SÍ", "SI")):
        return "SI"
    elif primera_linea.startswith("NO"):
        return "NO"
    else:
        return "NO DEFINIDO"

In [34]:
#path_general = 'C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones/'
#rut = '56589062'
#path_final = path_general + rut + '/transcriptions/'
# Ruta Transcripciones
#path = 'C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones/58211507/transcriptions/'

#Conviertiendo de JSON a TXT
#ruta_transcripcion_texto = convertir_json_a_texto_con_tiempo(path_final+"transription_0.json")

# Quitando Transcripción de Grabadora
#ruta_resultado_sin_grabadora = eliminar_frases_grabadora_archivo(ruta_transcripcion_texto, frases_grabadora)

#Limpiando Stopwords
#ruta_resultado_sin_stopwords = limpiar_stopwords_transcripcion(ruta_resultado_sin_grabadora)

#ruta_salida, segmentos = buscar_entrega_con_contexto(ruta_resultado_sin_stopwords,200)

#print("Rut :" + rut)

#respuesta = analizar_clave_con_gpt4all(ruta_salida)
#print("📌 Respuesta del modelo:")
#print(respuesta)

#respuesta_consolidada = extraer_respuesta_binaria(respuesta)
#print(respuesta_consolidada)

In [ ]:
import os
import pandas as pd

def procesar_grabaciones_y_guardar_excel(carpeta_general, ruta_excel_salida):
    """
    Recorre todas las subcarpetas de la carpeta de Grabaciones, ejecuta el pipeline
    de análisis y guarda las respuestas en un Excel consolidado.
    """

    registros = []

    for rut in os.listdir(carpeta_general):
        path_final = os.path.join(carpeta_general, rut, "transcriptions")

        # Verifica que exista el archivo de transcripción
        archivo_json = os.path.join(path_final, "transription_0.json")
        if not os.path.exists(archivo_json):
            print(f"⚠️ No se encontró transription_0.json para RUT {rut}, se omite.")
            continue

        try:
            # Paso a paso del procesamiento
            ruta_transcripcion_texto = convertir_json_a_texto_con_tiempo(archivo_json)
            ruta_sin_grabadora = eliminar_frases_grabadora_archivo(ruta_transcripcion_texto, frases_grabadora)
            ruta_sin_stopwords = limpiar_stopwords_transcripcion(ruta_sin_grabadora)
            ruta_segmentos, segmentos = buscar_entrega_con_contexto(ruta_sin_stopwords, 200)

            print(segmentos)-
            respuesta = analizar_clave_con_gpt4all(ruta_segmentos)
            binaria = extraer_respuesta_binaria(respuesta)

            registros.append({
                "RUT": rut,
                "Respuesta": binaria,
                "Explicación": respuesta,
                "Segmentos Detectados": len(segmentos)
            })

            print(f"✅ Procesado RUT {rut} - Respuesta: {binaria}")

        except Exception as e:
            print(f"❌ Error al procesar RUT {rut}: {e}")

    # Guardar Excel final
    df = pd.DataFrame(registros)
    df.to_excel(ruta_excel_salida, index=False)
    print(f"📁 Excel consolidado guardado en: {ruta_excel_salida}")

In [38]:
carpeta_base = "C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones"
excel_destino = "C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/resultados_fraude.xlsx"

procesar_grabaciones_y_guardar_excel(carpeta_base, excel_destino)

✅ Transcripción corregida y guardada en: C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones\58293900\transcriptions\transription_0_texto.txt
✅ Transcripción limpia guardada en: C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones\58293900\transcriptions\transription_0_texto_sin_grabadora.txt
✅ Transcripción sin stopwords guardada en: C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones\58293900\transcriptions\transription_0_texto_sin_grabadora_sin_stopwords.txt
✅ Segmentos guardados en: C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/Grabaciones\58293900\transcriptions\segmentos_entrega_contexto.txt
[]
Largo de la transcripción: 0 caracteres
🕒 Comienzo del análisis: 2025-04-08 21:45:28.249489
✅ Procesado RUT 58293900 - Respuesta: SI
📁 Excel consolidado guardado en: C:/Users/jmend/OneDrive/Escritorio/Magister/Capstone/NLP/resultados_fraude.xlsx
